<a href="https://colab.research.google.com/github/longjae/raft_rag/blob/master/noun_phrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [17]:
from tqdm import tqdm
import openai
from datasets import load_dataset
import random
import datasets
from huggingface_hub import login

In [4]:
# 1. Hugging Face에서 데이터셋 불러오기
dataset = load_dataset("longjae/klue-mrc-bge-m3")

# 2. train split을 데이터프레임으로 변환
df = dataset['train'].to_pandas()

# 3. 데이터프레임 확인
print(df.head())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/640 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/77.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10434 [00:00<?, ? examples/s]

                                         title news_category     source  \
0                       제주도 장마 시작 … 중부는 이달 말부터            종합   hankyung   
1      부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정            경제    acrofan   
2                                     나루세 요시히사          None  wikipedia   
3                               편의점 휩쓴 ‘맛집 라면’          생활경제   hankyung   
4  사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”            기획   hankyung   

                                             context  \
0  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...   
1  부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...   
2  시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...   
3  유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...   
4  “앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...   

                              question  question_type  is_impossible  \
0  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?              1          False   
1          지능형 생산자동화 기반기술을 개발중인 스타트업은?              2          False

In [5]:
# 임의 샘플링
df = df[1800:2000].reset_index(drop=True)

In [6]:
df

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples
0,"양파 값, 올초보다 50% 올라",생활경제,hankyung,지난달 배추 무 소고기 가격은 하락하고 양파값은 반등했다.한국소비자원은 생필품 가격...,1월보다 13.7%하락한 가격으로 판매되는 신선식품은?,2,False,배추,157,[이달 들어 찾아온 한파와 폭설로 채소값이 크게 오르고 있다. 특히 기온 변화에 민...
1,"市, 조속히 재개장 결정 내려달라”",전국,hankyung,“제2롯데월드 내 영화관과 수족관 영업이 3개월 넘게 중단되면서 지역경제에 타격을 ...,송파구청장이 사시에 합격한 연도는?,2,False,2002년,228,[충청남도 공주시 출신으로 서울 휘문고등학교와 서울대학교 법학과를 졸업했다. 일선 ...
2,"CJ프레시웨이, 국내산 샤인머스캣 중국 시장 진출에 힘 보탠다",경제,acrofan,"CJ프레시웨이(대표이사 문종석)가 지자체, 농가와 손잡고 명품포도로 주목받고 있는 ...",샤인 머스캣의 별명은 무엇인가?,1,False,‘망고 포도’,307,"[“꿈과 관련된 노래가 정말 좋고, 샤이니의 꿈 같은 이미지를 심어줄 수 있을 것 ..."
3,"정통 수제 싱글몰트 위스키 발베니, ‘발베니 스토리 전시회’ 개최",산업,acrofan,윌리엄그랜트앤선즈코리아의 정통 수제 싱글몰트 위스키 발베니가 서울 성동구에 위치한 ...,발베니 스토리 전시회에 도서 300여 권을 제공하는 서점이 있는 도시는?,1,False,서울,567,[독서의 계절 가을에 어울리는 책 축제가 잇달아 열린다. 1일 서울 서교동 일대에서...
4,"주말, 꽃도 보고 집도 보고",부동산,hankyung,“봄 축제도 즐기고 모델하우스도 둘러보세요.” 전국 봄꽃 축제가 본격화한 가운데 축...,안산벚꽃음악회가 열리는 인근에서 홍보 중인 아파트는?,1,False,‘e편한세상 신촌’,294,[‘8·28 전·월세 대책’ 발표 이후 분양시장이 활기를 띠자 건설사들이 실수요자들...
...,...,...,...,...,...,...,...,...,...,...
195,신동빈 회장 “제2 롯데월드 안전에 최우선”...권오준 회장 “계열사 매각 등 구조...,산업섹션,hankyung,신동빈 롯데그룹 회장과 권오준 포스코 회장 등 주요 대기업 총수와 최고경영자(CEO...,금호아시아나그룹의 경쟁 회사는?,2,False,대한항공,878,[공기업과 대기업의 채용이 잇따르고 있다. 금호아시아나그룹은 21일부터 다음달 7일...
196,"이노비즈協, 혁신기업 러시아 스타트업 진출 전략 워크숍 개최",경제,acrofan,"중소벤처기업부(장관 홍종학)와 이노비즈협회(중소기업기술혁신협회, 회장 조홍래)는 4...","워크숍에서, 러시아 창업행사에 참여할 수 있는 기업은 몇 개가 선발될 예정인가?",1,False,10개,823,"[20개국 4,000여명의 창업가와 투자자들이 참여한 서울시 최초의 글로벌 스타트업..."
197,서울 개업 변호사 1만명 … ‘쏠림’ 심화,사회,hankyung,서울에서 개업한 변호사 수가 지난해 처음으로 1만명을 돌파했다. 대한변호사협회는 지...,서울 지역의 개업변호사가 1만474명이었던 시기에 마을변호사는 몇 명이었는가?,2,False,505명,715,[전국 최대 변호사 밀집지역인 서울 서초동 법조타운에서 일해온 홍승권 변호사(30·...
198,하버드의 계산수,None,wikipedia,하버드의 계산수들(Harvard Computers)은 미국 천문학자 에드워드 찰스 ...,피커링이 최초로 업무를 시키기 위해 뽑은 여성의 이전 직업은?,1,False,하녀,314,"[1918년 1월 핀란드 내전이 시작되자 적위대는 여성들을 동원해 간호, 요리, 옷..."


In [7]:
system_prompt = """당신은 주어진 질문을 명사구 형태로 바꿔서 재작성해야 합니다.

### 지시사항
1. 주어진 질문을 명사구 형태로 바꾸세요.
2. 명사구 형태로 바꿀 때 정보의 손실은 없었으면 합니다.

예시)
질문: 서울 지역의 개업변호사가 1만474명이었던 시기에 마을변호사는 몇 명이었는가?
재작성: 서울 지역의 개업변호사가 1만474명이었던 시기의 마을변호사 인원수

질문: 발베니 스토리 전시회에 도서 300여 권을 제공하는 서점이 있는 도시는?
재작성: 발베니 스토리 전시회에 도서 300여 권을 제공하는 서점이 있는 도시"""

In [8]:
questions = df["question"].to_list()

In [9]:
user_prompts = []

for question in questions:
  user_prompts.append("질문: " + question + "\n재작성:")

In [10]:
print(user_prompts[0])

질문: 1월보다 13.7%하락한 가격으로 판매되는 신선식품은?
재작성:


In [11]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [13]:
# 5개로 테스트
result_lst = []
for user_prompt in tqdm(user_prompts[:5]):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt}
      ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 5/5 [00:03<00:00,  1.32it/s]


In [14]:
user_prompts[:5]

['질문: 1월보다 13.7%하락한 가격으로 판매되는 신선식품은?\n재작성:',
 '질문: 송파구청장이 사시에 합격한 연도는?\n재작성:',
 '질문: 샤인 머스캣의 별명은 무엇인가?\n재작성:',
 '질문: 발베니 스토리 전시회에 도서 300여 권을 제공하는 서점이 있는 도시는?\n재작성:',
 '질문: 안산벚꽃음악회가 열리는 인근에서 홍보 중인 아파트는?\n재작성:']

In [15]:
result_lst

['1월보다 13.7% 하락한 가격으로 판매되는 신선식품',
 '송파구청장이 사시에 합격한 연도',
 '샤인 머스캣의 별명',
 '발베니 스토리 전시회에 도서 300여 권을 제공하는 서점이 있는 도시',
 '안산벚꽃음악회가 열리는 인근에서 홍보 중인 아파트']

In [16]:
result_lst = []
for user_prompt in tqdm(user_prompts):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt}
      ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 200/200 [02:43<00:00,  1.22it/s]


In [18]:
df['question'] = result_lst

In [19]:
df

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples
0,"양파 값, 올초보다 50% 올라",생활경제,hankyung,지난달 배추 무 소고기 가격은 하락하고 양파값은 반등했다.한국소비자원은 생필품 가격...,1월보다 13.7% 하락한 가격으로 판매되는 신선식품,2,False,배추,157,[이달 들어 찾아온 한파와 폭설로 채소값이 크게 오르고 있다. 특히 기온 변화에 민...
1,"市, 조속히 재개장 결정 내려달라”",전국,hankyung,“제2롯데월드 내 영화관과 수족관 영업이 3개월 넘게 중단되면서 지역경제에 타격을 ...,송파구청장이 사시에 합격한 연도,2,False,2002년,228,[충청남도 공주시 출신으로 서울 휘문고등학교와 서울대학교 법학과를 졸업했다. 일선 ...
2,"CJ프레시웨이, 국내산 샤인머스캣 중국 시장 진출에 힘 보탠다",경제,acrofan,"CJ프레시웨이(대표이사 문종석)가 지자체, 농가와 손잡고 명품포도로 주목받고 있는 ...",샤인 머스캣의 별명,1,False,‘망고 포도’,307,"[“꿈과 관련된 노래가 정말 좋고, 샤이니의 꿈 같은 이미지를 심어줄 수 있을 것 ..."
3,"정통 수제 싱글몰트 위스키 발베니, ‘발베니 스토리 전시회’ 개최",산업,acrofan,윌리엄그랜트앤선즈코리아의 정통 수제 싱글몰트 위스키 발베니가 서울 성동구에 위치한 ...,발베니 스토리 전시회에 도서 300여 권을 제공하는 서점이 있는 도시,1,False,서울,567,[독서의 계절 가을에 어울리는 책 축제가 잇달아 열린다. 1일 서울 서교동 일대에서...
4,"주말, 꽃도 보고 집도 보고",부동산,hankyung,“봄 축제도 즐기고 모델하우스도 둘러보세요.” 전국 봄꽃 축제가 본격화한 가운데 축...,안산벚꽃음악회가 열리는 인근에서 홍보 중인 아파트,1,False,‘e편한세상 신촌’,294,[‘8·28 전·월세 대책’ 발표 이후 분양시장이 활기를 띠자 건설사들이 실수요자들...
...,...,...,...,...,...,...,...,...,...,...
195,신동빈 회장 “제2 롯데월드 안전에 최우선”...권오준 회장 “계열사 매각 등 구조...,산업섹션,hankyung,신동빈 롯데그룹 회장과 권오준 포스코 회장 등 주요 대기업 총수와 최고경영자(CEO...,금호아시아나그룹의 경쟁 회사,2,False,대한항공,878,[공기업과 대기업의 채용이 잇따르고 있다. 금호아시아나그룹은 21일부터 다음달 7일...
196,"이노비즈協, 혁신기업 러시아 스타트업 진출 전략 워크숍 개최",경제,acrofan,"중소벤처기업부(장관 홍종학)와 이노비즈협회(중소기업기술혁신협회, 회장 조홍래)는 4...",워크숍에서 러시아 창업행사에 참여할 수 있는 기업의 선발 예정 개수,1,False,10개,823,"[20개국 4,000여명의 창업가와 투자자들이 참여한 서울시 최초의 글로벌 스타트업..."
197,서울 개업 변호사 1만명 … ‘쏠림’ 심화,사회,hankyung,서울에서 개업한 변호사 수가 지난해 처음으로 1만명을 돌파했다. 대한변호사협회는 지...,서울 지역의 개업변호사가 1만474명이었던 시기의 마을변호사 인원수,2,False,505명,715,[전국 최대 변호사 밀집지역인 서울 서초동 법조타운에서 일해온 홍승권 변호사(30·...
198,하버드의 계산수,None,wikipedia,하버드의 계산수들(Harvard Computers)은 미국 천문학자 에드워드 찰스 ...,피커링이 최초로 업무를 시키기 위해 뽑은 여성의 이전 직업,1,False,하녀,314,"[1918년 1월 핀란드 내전이 시작되자 적위대는 여성들을 동원해 간호, 요리, 옷..."


In [20]:
# 정답이 첫번째 문서에만 위치하지 않도록 문서를 섞는다.
samples = []
for context, negative_samples in zip(df['context'].to_list(), df['negative_samples'].to_list()):
  documents = [context] + list(negative_samples)
  random.shuffle(documents)
  samples.append(documents)

In [21]:
# 프롬프트에 검색 문서 넣는 연습
for idx, s in enumerate(samples[5]):
  print('문서' + str(idx + 1) + ':' + s)
  print('---')

문서1:진실규명 결정을 받은 김지태의 유가족들은 2010년 6월에야 법원에 정수장학회와 국가를 상대로 낸 주식양도 등 청구소송을 냈다. 김씨 측은 "박 전 대통령이 사망하고 난 이후 1980년에 토지 반환청구 의사를 표시했고, 과거사정리위원회의 진실규명 결정을 송달받은 이후 손해배상을 청구한 것이므로 공소시효가 남아있다"고 주장했다.

하지만 1심 재판부는 "소멸시효가 지났다"며 김씨 측의 청구를 기각했고, 2심 재판부도 김씨가 국가의 강박행위로 인해 재산을 헌납한 것은 인정하면서도 의사결정권이 완전히 박탈당한 상태는 아니었던 것으로 판단해 원고 패소 판결했다. 2012년 2월 24일 서울중앙지법 민사합의17부(재판장 염원섭)에 의해 5.16장학회의 ‘헌납’ 과정에서 강압이 있었음이 다시 한 번 입증되었다. 하지만 재판부는 김영우가 제기한 과거 부일장학회의 주식반환에 대해서는 공소시효 소멸을 이유로 기각하였다. 이에 국가의 범죄에 대해서는 공소 시효의 범위를 폭넓게 인정해줘야 한다는 비판도 제기되었다. 

대법원은 2014년 2월 13일 김지태씨 장남 영구 씨를 비롯한 유가족 6명이 정수장학회와 국가를 상대로 낸 주식양도 등 청구소송 상고심에서 심리불속행 기각 결정을 내렸다. '심리불속행'은 상고 사건 가운데 상고 대상이 아니라고 판단되는 사건은 더이상 심리하지 않고 기각하는 제도다.
---
문서2:그는 구두로는 안나 니콜 스미스에게 자신의 재산의 반을 주겠다고 약속했다 하나 유언장에는 안나 니콜 스미스의 이름은 없었다. 그가 죽고 안나 니콜 스미스는 하워드 마셜의 상속인인 피어스 마샬과 12년간 법정 소송을 하였다.

하워드 마셜은 유언장에서 유산 상속자로 안나 니콜 스미스와 자신의 다른 아들 제임스 하워드 마샬 3세, 그밖의 자녀들은 없었고, 자신의 사업 동반자이자 후계자로 지목한 에버렛 피어스 마샬을 지목했다. 안나 니콜 스미스와 다른 아들 제임스 하워드 마샬 3세는 유산을 획득하기 위해 미국 내 모든 주의 법원과 미국 연방 법원에서 소송을 걸었지만 텍사

In [22]:
user_prompts = []
for sample in samples:
  user_prompt = '검색 결과:\n'
  for idx, s in enumerate(sample):
    user_prompt = user_prompt  + '문서' + str(idx + 1) + ':' + s + '\n'
    user_prompt = user_prompt + '---\n'
  user_prompts.append(user_prompt)

In [23]:
system_prompt = """당신은 검색된 문서부터 질문의 답변을 작성하는 언어 모델입니다.

### 지시사항
1. 질문으로부터 주어진 다수의 문서에서 답을 찾아 작성하세요.
2. 검색된 문서에 질문의 답이 없는 경우에는 질문에 대한 내용을 언급하면서 답을 찾을 수 없다고 작성하세요.
3. 답변에서 참고자료의 내용을 인용한 경우, 답변 맨 끝에 인용한 문서 id를 추가하세요. 답변에 내용을 인용한 경우에만 작성해야 합니다. 인용한 게 없다면 쓰지마세요.
4. 문서 id 값은 검색 결과에서 가져온 것이며 두 개의 대괄호로 묶어야 하며 ref라는 키워드와 함께 작성해야 합니다. 예를 들어 특정 문단이 3번, 5번 문서에서 인용했다면 다음과 같이 작성하십시오.
예시) ~~~해서 했습니다. [[ref3]], [[ref5]]
5. 주요 문장 또는 문단 뒤에 ref문서번호를 덧붙이는 것은 매우 중요합니다. 이는 반드시 지켜져야 합니다.
6. 문단 중간 중간에 인용이 들어가더라도 답변은 하나의 이어지는 평문으로 작성하십시오.
7. 주어진 문서는 검색 결과입니다. 따라서 질문과 연관없는 문서가 숨어있을 수 있으니 이를 인용하지 않도록 주의하십시오.
8. 검색 결과에 없는 내용은 답변으로 작성하지 마십시오. 가장 중요한 지시사항입니다. 따라서 인용 없이는 그 어떠한 설명도 제공할 수 없습니다.
9. 답변은 최대한 풍부하게 작성해주시기 바랍니다.
10. 검색 결과는 모두 독립적인 문서들이며 반드시 연관되어져 있거나 순서대로 이어지는 문서들이 아닙니다.
"""

In [24]:
user_prompt_for_answer = []
for multi_document_question, user_prompt in zip(df['question'].to_list(), user_prompts):
  user_prompt_for_answer.append('질문:' + multi_document_question + '\n' + user_prompt + '답변:')

In [25]:
print(user_prompt_for_answer[5])

질문:하워드 마셜의 재산 상속을 위해 안나 니콜 스미스가 법정 소송을 벌인 기간
검색 결과:
문서1:진실규명 결정을 받은 김지태의 유가족들은 2010년 6월에야 법원에 정수장학회와 국가를 상대로 낸 주식양도 등 청구소송을 냈다. 김씨 측은 "박 전 대통령이 사망하고 난 이후 1980년에 토지 반환청구 의사를 표시했고, 과거사정리위원회의 진실규명 결정을 송달받은 이후 손해배상을 청구한 것이므로 공소시효가 남아있다"고 주장했다.

하지만 1심 재판부는 "소멸시효가 지났다"며 김씨 측의 청구를 기각했고, 2심 재판부도 김씨가 국가의 강박행위로 인해 재산을 헌납한 것은 인정하면서도 의사결정권이 완전히 박탈당한 상태는 아니었던 것으로 판단해 원고 패소 판결했다. 2012년 2월 24일 서울중앙지법 민사합의17부(재판장 염원섭)에 의해 5.16장학회의 ‘헌납’ 과정에서 강압이 있었음이 다시 한 번 입증되었다. 하지만 재판부는 김영우가 제기한 과거 부일장학회의 주식반환에 대해서는 공소시효 소멸을 이유로 기각하였다. 이에 국가의 범죄에 대해서는 공소 시효의 범위를 폭넓게 인정해줘야 한다는 비판도 제기되었다. 

대법원은 2014년 2월 13일 김지태씨 장남 영구 씨를 비롯한 유가족 6명이 정수장학회와 국가를 상대로 낸 주식양도 등 청구소송 상고심에서 심리불속행 기각 결정을 내렸다. '심리불속행'은 상고 사건 가운데 상고 대상이 아니라고 판단되는 사건은 더이상 심리하지 않고 기각하는 제도다.
---
문서2:그는 구두로는 안나 니콜 스미스에게 자신의 재산의 반을 주겠다고 약속했다 하나 유언장에는 안나 니콜 스미스의 이름은 없었다. 그가 죽고 안나 니콜 스미스는 하워드 마셜의 상속인인 피어스 마샬과 12년간 법정 소송을 하였다.

하워드 마셜은 유언장에서 유산 상속자로 안나 니콜 스미스와 자신의 다른 아들 제임스 하워드 마샬 3세, 그밖의 자녀들은 없었고, 자신의 사업 동반자이자 후계자로 지목한 에버렛 피어스 마샬을 지목했다. 안나 니콜 스미스와 다른 아들 제임스 하워드 마샬 3

In [26]:
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 200/200 [08:05<00:00,  2.43s/it]


In [27]:
import re

In [28]:
# 인용 문서 번호 추출
def extract_ref_numbers(text):
    # 정규표현식을 사용하여 [[ref숫자]] 패턴 찾기
    pattern = r'\[\[ref(\d+)\]\]'

    # 모든 매치 찾기
    matches = re.findall(pattern, text)

    # 문자열을 정수로 변환하고 중복 제거 후 정렬
    unique_numbers = sorted(set(map(int, matches)))

    return unique_numbers

In [29]:
result_lst[0]

'1월보다 13.7% 하락한 가격으로 판매되는 신선식품은 배추입니다. 배추의 평균 판매가는 1월 1886원이었으나, 최근에는 1402원으로 하락하여 13.7% 감소한 것으로 나타났습니다 [[ref2]].'

In [30]:
# 테스트
result = extract_ref_numbers(result_lst[0])
print(result)

[2]


In [31]:
extracted_ref_numbers = []
for sample in result_lst:
  extracted_ref_numbers.append(extract_ref_numbers(sample))

In [32]:
# 인용 번호 없는 샘플 찾아내기
for idx, extracted_ref_number in enumerate(extracted_ref_numbers):
  if len(extracted_ref_number) == 0:
    print(idx)
    print(result_lst[idx])
    print('--')

28
검색된 문서들에서는 궁전 아래에 위치한 집의 방 칸 수에 대한 구체적인 정보가 포함되어 있지 않습니다. 따라서 궁전 아래에 위치하던 집의 방 칸 수에 대한 답변을 찾을 수 없습니다.
--
62
검색된 문서들에서는 헬레네 바이겔의 남편에 대한 정보가 포함되어 있지 않습니다. 따라서 헬레네 바이겔의 남편에 대한 내용을 찾을 수 없습니다.
--
69
화재조사강사 자격 시험을 주관하는 기관에 대한 정보는 검색된 문서에서 확인할 수 없었습니다. 따라서 이 질문에 대한 답변을 제공할 수 없습니다.
--
84
8월 경상수지에 대한 구체적인 정보는 검색된 문서에서 확인할 수 없었습니다. 따라서 8월 경상수지 흑자에 대한 내용을 제공할 수 없습니다.
--
102
복지부가 예비타당성 절차 없이 시행을 요구한 사업에 대한 구체적인 출처는 검색된 문서에서 확인할 수 없습니다. 그러나 문서3에서는 복지부가 저소득층 기저귀·분유 지원 사업에 대해 예비타당성 조사를 받기로 했다는 내용이 있으며, 복지부가 예비타당성 조사를 면제받고자 했던 요청이 기재부에 의해 수용되지 않았다는 점이 언급되고 있습니다. 이와 같은 사례는 복지부가 예비타당성 절차를 요구하는 사업에 대해 신중하게 접근하고 있음을 보여줍니다. 하지만 복지부가 예비타당성 절차 없이 시행을 요구한 특정 사업에 대한 정보는 제공되지 않았습니다. 따라서 질문에 대한 명확한 답변을 찾을 수 없었습니다.
--
114
검색된 문서들에서는 "이전에 개설된 해외 법인 계좌 중, 국세청에 공유되지 않을 예정인 금액 미만의 금액"에 대한 구체적인 정보가 포함되어 있지 않습니다. 따라서 해당 질문에 대한 답변을 찾을 수 없습니다.
--
148
검색된 문서들에서는 2008년 설립된 광고 업체의 세계 지사 수에 대한 구체적인 정보가 포함되어 있지 않습니다. 따라서 해당 질문에 대한 답변을 찾을 수 없습니다.
--
163
검색된 문서들에서는 보닌의 지인 중 마술사의 나이에 대한 정보가 포함되어 있지 않습니다. 따라서 이 질문에 대한 답변을 찾을 수

In [33]:
df['search_result'] = samples

In [34]:
df['answer'] = result_lst

In [35]:
df['extracted_ref_numbers'] = extracted_ref_numbers

In [36]:
# 인용 실패한 데이터 제외
cited_df = df[~(df['extracted_ref_numbers'].apply(str) == '[]')].reset_index(drop=True)

In [37]:
print('답변이 존재하는 데이터:', len(cited_df))

답변이 존재하는 데이터: 189


In [38]:
cited_df[['question', 'search_result', 'answer', 'extracted_ref_numbers']]

,question,search_result,answer,extracted_ref_numbers
0,1월보다 13.7% 하락한 가격으로 판매되는 신선식품,[마켓컬리는 식탁의 기본이 되는 신선식품을 합리적인 가격으로 안전성까지 검증해 판매...,1월보다 13.7% 하락한 가격으로 판매되는 신선식품은 배추입니다. 배추의 평균 판...,[2]
1,송파구청장이 사시에 합격한 연도,[경기도 광주 출신으로 일본에 유학하여 호세이 대학의 전신인 도쿄의 와후쓰 법률학교...,송파구청장 박춘희는 2002년에 사법고시에 합격하였습니다. 그는 49세의 나이에 사...,[4]
2,샤인 머스캣의 별명,"[매그니토(Magneto, 본명: 막스 아이젠하르트, Max Eisenhardt )...",샤인 머스캣은 '망고 포도'라는 별명으로도 알려져 있습니다. 이는 샤인 머스캣이 청...,[4]
3,발베니 스토리 전시회에 도서 300여 권을 제공하는 서점이 있는 도시,[윌리엄그랜트앤선즈코리아의 정통 수제 싱글몰트 위스키 발베니가 서울 성동구에 위치한...,발베니 스토리 전시회에 도서 300여 권을 제공하는 서점이 있는 도시는 서울입니다....,[1]
4,안산벚꽃음악회가 열리는 인근에서 홍보 중인 아파트,[“봄 축제도 즐기고 모델하우스도 둘러보세요.” 전국 봄꽃 축제가 본격화한 가운데 ...,안산벚꽃음악회가 열리는 인근에서 홍보 중인 아파트는 대림산업이 분양하는 ‘e편한세상...,[1]
...,...,...,...,...
184,금호아시아나그룹의 경쟁 회사,[공기업과 대기업의 채용이 잇따르고 있다. 금호아시아나그룹은 21일부터 다음달 7일...,금호아시아나그룹의 주요 경쟁 회사로는 대한항공이 있습니다. 대한항공은 금호아시아나그...,"[1, 4]"
185,워크숍에서 러시아 창업행사에 참여할 수 있는 기업의 선발 예정 개수,[정부가 향후 5년간 40조원의 예산을 창조경제 분야에 투입해 65만개의 신규 일자...,워크숍에서 러시아 창업행사에 참여할 수 있는 기업의 선발 예정 개수는 10개사입니다...,[4]
186,서울 지역의 개업변호사가 1만474명이었던 시기의 마을변호사 인원수,[서울에서 개업한 변호사 수가 지난해 처음으로 1만명을 돌파했다. 대한변호사협회는 ...,서울 지역의 개업 변호사가 1만474명이었던 시기에 마을변호사 인원수는 505명이었...,[1]
187,피커링이 최초로 업무를 시키기 위해 뽑은 여성의 이전 직업,[아그노디케는 고대 아네나이 사람으로 당시로서는 여성에게 금지되어 있던 의술을 배웠...,"피커링이 최초로 업무를 시키기 위해 뽑은 여성은 윌리어미나 플레밍으로, 그녀는 원래...",[4]


In [39]:
from google.colab import userdata
HF_WRITE_API_KEY = userdata.get('HF_WRITE_API_KEY')

login(HF_WRITE_API_KEY)

In [40]:
dataset = datasets.Dataset.from_pandas(cited_df)

In [41]:
dataset.push_to_hub("klue-mrc-paraphrased-question")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/longjae/klue-mrc-paraphrased-question/commit/b08bb0d76b192ad0adb829f5fd5943a78df1ed18', commit_message='Upload dataset', commit_description='', oid='b08bb0d76b192ad0adb829f5fd5943a78df1ed18', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/longjae/klue-mrc-paraphrased-question', endpoint='https://huggingface.co', repo_type='dataset', repo_id='longjae/klue-mrc-paraphrased-question'), pr_revision=None, pr_num=None)

In [42]:
cited_df

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples,search_result,answer,extracted_ref_numbers
0,"양파 값, 올초보다 50% 올라",생활경제,hankyung,지난달 배추 무 소고기 가격은 하락하고 양파값은 반등했다.한국소비자원은 생필품 가격...,1월보다 13.7% 하락한 가격으로 판매되는 신선식품,2,False,배추,157,[이달 들어 찾아온 한파와 폭설로 채소값이 크게 오르고 있다. 특히 기온 변화에 민...,[마켓컬리는 식탁의 기본이 되는 신선식품을 합리적인 가격으로 안전성까지 검증해 판매...,1월보다 13.7% 하락한 가격으로 판매되는 신선식품은 배추입니다. 배추의 평균 판...,[2]
1,"市, 조속히 재개장 결정 내려달라”",전국,hankyung,“제2롯데월드 내 영화관과 수족관 영업이 3개월 넘게 중단되면서 지역경제에 타격을 ...,송파구청장이 사시에 합격한 연도,2,False,2002년,228,[충청남도 공주시 출신으로 서울 휘문고등학교와 서울대학교 법학과를 졸업했다. 일선 ...,[경기도 광주 출신으로 일본에 유학하여 호세이 대학의 전신인 도쿄의 와후쓰 법률학교...,송파구청장 박춘희는 2002년에 사법고시에 합격하였습니다. 그는 49세의 나이에 사...,[4]
2,"CJ프레시웨이, 국내산 샤인머스캣 중국 시장 진출에 힘 보탠다",경제,acrofan,"CJ프레시웨이(대표이사 문종석)가 지자체, 농가와 손잡고 명품포도로 주목받고 있는 ...",샤인 머스캣의 별명,1,False,‘망고 포도’,307,"[“꿈과 관련된 노래가 정말 좋고, 샤이니의 꿈 같은 이미지를 심어줄 수 있을 것 ...","[매그니토(Magneto, 본명: 막스 아이젠하르트, Max Eisenhardt )...",샤인 머스캣은 '망고 포도'라는 별명으로도 알려져 있습니다. 이는 샤인 머스캣이 청...,[4]
3,"정통 수제 싱글몰트 위스키 발베니, ‘발베니 스토리 전시회’ 개최",산업,acrofan,윌리엄그랜트앤선즈코리아의 정통 수제 싱글몰트 위스키 발베니가 서울 성동구에 위치한 ...,발베니 스토리 전시회에 도서 300여 권을 제공하는 서점이 있는 도시,1,False,서울,567,[독서의 계절 가을에 어울리는 책 축제가 잇달아 열린다. 1일 서울 서교동 일대에서...,[윌리엄그랜트앤선즈코리아의 정통 수제 싱글몰트 위스키 발베니가 서울 성동구에 위치한...,발베니 스토리 전시회에 도서 300여 권을 제공하는 서점이 있는 도시는 서울입니다....,[1]
4,"주말, 꽃도 보고 집도 보고",부동산,hankyung,“봄 축제도 즐기고 모델하우스도 둘러보세요.” 전국 봄꽃 축제가 본격화한 가운데 축...,안산벚꽃음악회가 열리는 인근에서 홍보 중인 아파트,1,False,‘e편한세상 신촌’,294,[‘8·28 전·월세 대책’ 발표 이후 분양시장이 활기를 띠자 건설사들이 실수요자들...,[“봄 축제도 즐기고 모델하우스도 둘러보세요.” 전국 봄꽃 축제가 본격화한 가운데 ...,안산벚꽃음악회가 열리는 인근에서 홍보 중인 아파트는 대림산업이 분양하는 ‘e편한세상...,[1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,신동빈 회장 “제2 롯데월드 안전에 최우선”...권오준 회장 “계열사 매각 등 구조...,산업섹션,hankyung,신동빈 롯데그룹 회장과 권오준 포스코 회장 등 주요 대기업 총수와 최고경영자(CEO...,금호아시아나그룹의 경쟁 회사,2,False,대한항공,878,[공기업과 대기업의 채용이 잇따르고 있다. 금호아시아나그룹은 21일부터 다음달 7일...,[공기업과 대기업의 채용이 잇따르고 있다. 금호아시아나그룹은 21일부터 다음달 7일...,금호아시아나그룹의 주요 경쟁 회사로는 대한항공이 있습니다. 대한항공은 금호아시아나그...,"[1, 4]"
185,"이노비즈協, 혁신기업 러시아 스타트업 진출 전략 워크숍 개최",경제,acrofan,"중소벤처기업부(장관 홍종학)와 이노비즈협회(중소기업기술혁신협회, 회장 조홍래)는 4...",워크숍에서 러시아 창업행사에 참여할 수 있는 기업의 선발 예정 개수,1,False,10개,823,"[20개국 4,000여명의 창업가와 투자자들이 참여한 서울시 최초의 글로벌 스타트업...",[정부가 향후 5년간 40조원의 예산을 창조경제 분야에 투입해 65만개의 신규 일자...,워크숍에서 러시아 창업행사에 참여할 수 있는 기업의 선발 예정 개수는 10개사입니다...,[4]
186,서울 개업 변호사 1만명 … ‘쏠림’ 심화,사회,hankyung,서울에서 개업한 변호사 수가 지난해 처음으로 1만명을 돌파했다. 대한변호사협회는 지...,서울 지역의 개업변호사가 1만474명이었던 시기의 마을변호사 인원수,2,False,505명,715,[전국 최대 변호사 밀집지역인 서울 서초동 법조타운에서 일해온 홍승권 변호사(30·...,[서울에서 개업한 변호사 수가 지난해 처음으로 1만명을 돌파했다. 대한변호사협회는 ...,서울 지역의 개업 변호사가 1만474명이었던 시기에 마을변호사 인원수는 505명이었...,[1]
187,하버드의 계산수,None,wikipedia,하버드의 계산수들(Harvard Computers)은 미국 천문학자 에드워드 찰스 ...,피커링이 최초로 업무를 시키기 위해 뽑은 여성의 이전 직업,1,False,하녀,314,"[1918년 1월 핀란드 내전이 시작되자 적위대는 여성들을 동원해 간호, 요리, 옷...",[아그노디케는 고대 아네나이 사람으로 당시로서는 여성에게 금지되어 있던 의술을 배웠...,"피커링이 최초로 업무를 시키기 위해 뽑은 여성은 윌리어미나 플레밍으로, 그녀는 원래...",[4]
